#  COMP3210/6210: Assignment 2

## **Task 1:** Nearest Neighbor Search (refer to weeks 8 & 9 & 10 lecture notes)

###   Select ONE dataset (Restaurant, Shop, or Parking).

In [9]:
# Importing necessary library
import os

# Define the file path
file_path = os.path.expanduser('~/mol/data/assignment-2-datasets-1 (1)/Task1_Datasets/restaurant_dataset.txt')

# Function to read the dataset from a text file
def read_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            point_id = int(parts[0])
            x = float(parts[1])
            y = float(parts[2])
            data.append((point_id, x, y))
    return data

# Read the restaurant dataset
restaurant_data = read_dataset(file_path)

# Display the contents of the dataset
for point in restaurant_data:
    print(point)


(1, 29.5, 91.46)
(2, 59.24, 124.25)
(3, 35.08, 147.04)
(4, 75.62, 76.66)
(5, 11.0, 110.78)
(6, 0.69, 128.89)
(7, 59.11, 166.68)
(8, 21.45, 74.95)
(9, 5.69, 112.42)
(10, 55.95, 104.68)
(11, 6.46, 131.41)
(12, 63.45, 138.92)
(13, 26.67, 129.78)
(14, 68.03, 143.89)
(15, 18.92, 140.31)
(16, 55.19, 80.19)
(17, 25.41, 120.19)
(18, 50.27, 159.55)
(19, 66.69, 76.29)
(20, 82.62, 159.61)
(21, 41.43, 129.85)
(22, 46.18, 104.65)
(23, 31.47, 79.69)
(24, 44.67, 131.25)
(25, 85.13, 111.12)
(26, 54.57, 97.28)
(27, 24.75, 128.53)
(28, 24.4, 155.03)
(29, 17.46, 116.25)
(30, 75.28, 84.62)
(31, 67.02, 139.32)
(32, 21.2, 158.66)
(33, 69.84, 158.4)
(34, 67.57, 78.15)
(35, 1.03, 121.0)
(36, 34.3, 153.63)
(37, 27.9, 128.49)
(38, 14.92, 151.58)
(39, 46.37, 73.88)
(40, 24.61, 123.95)
(41, 67.94, 110.21)
(42, 41.55, 84.78)
(43, 40.91, 98.2)
(44, 16.91, 132.13)
(45, 39.96, 150.59)
(46, 86.37, 124.56)
(47, 58.93, 149.97)
(48, 51.48, 92.7)
(49, 16.6, 101.69)
(50, 64.31, 111.23)
(51, 51.11, 120.2)
(52, 7.55, 89.62)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [7]:
import os
import math
from rtree import index

# Function to read data from a text file
def read_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            point_id = int(parts[0])
            x = float(parts[1])
            y = float(parts[2])
            data.append((point_id, x, y))
    return data

# Function to compute Euclidean distance between two points
def euclidean_distance(point1, point2):
    return math.sqrt((point1[1] - point2[1]) ** 2 + (point1[2] - point2[2]) ** 2)

# Function to find the nearest neighbor for each query point in a dataset using Sequential Scan
def sequential_scan_nearest_neighbors(queries, dataset):
    nearest_neighbors = []
    for query in queries:
        min_distance = float('inf')
        nearest_point = None
        for point in dataset:
            distance = euclidean_distance(query, point)
            if distance < min_distance:
                min_distance = distance
                nearest_point = point
        nearest_neighbors.append((query[0], nearest_point[0], min_distance))
    return nearest_neighbors

# Function to create an R-tree index from a dataset
def create_rtree_index(dataset):
    p = index.Property()
    idx = index.Index(properties=p)
    for point in dataset:
        idx.insert(point[0], (point[1], point[2], point[1], point[2]), obj=point)
    return idx

# Function to find the nearest neighbor for each query point using the Best First algorithm with R-tree
def best_first_nearest_neighbors(queries, rtree_index):
    nearest_neighbors = []
    for query in queries:
        nearest = list(rtree_index.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        distance = euclidean_distance(query, nearest)
        nearest_neighbors.append((query[0], nearest[0], distance))
    return nearest_neighbors

# Function to divide the dataset into two subspaces
def divide_dataset(dataset, dimension='x'):
    dataset_sorted = sorted(dataset, key=lambda point: point[1] if dimension == 'x' else point[2])
    mid = len(dataset_sorted) // 2
    return dataset_sorted[:mid], dataset_sorted[mid:]

# Function to find the nearest neighbor for each query point using BF with Divide-and-Conquer
def bf_divide_and_conquer_nearest_neighbors(queries, dataset):
    subspace1, subspace2 = divide_dataset(dataset)
    rtree_index1 = create_rtree_index(subspace1)
    rtree_index2 = create_rtree_index(subspace2)
    
    nearest_neighbors = []
    for query in queries:
        nearest1 = list(rtree_index1.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        nearest2 = list(rtree_index2.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        distance1 = euclidean_distance(query, nearest1)
        distance2 = euclidean_distance(query, nearest2)
        if distance1 < distance2:
            nearest_neighbors.append((query[0], nearest1[0], distance1))
        else:
            nearest_neighbors.append((query[0], nearest2[0], distance2))
    return nearest_neighbors

# Paths to dataset files
restaurant_file = os.path.expanduser('~/mol/data/assignment-2-datasets-1 (1)/Task1_Datasets/restaurant_dataset.txt')
queries_file = os.path.expanduser('~/mol/data/assignment-2-datasets-1 (1)/Task1_Datasets/query_points.txt')

# Read datasets and queries
restaurant_data = read_dataset(restaurant_file)
user_queries = read_dataset(queries_file)

# Sequential Scan
nearest_restaurants_seq = sequential_scan_nearest_neighbors(user_queries, restaurant_data)
print("Nearest Restaurants using Sequential Scan:")
for result in nearest_restaurants_seq:
    print(f"User {result[0]} -> Restaurant {result[1]} (Distance: {result[2]:.2f})")

# Best First with R-tree
restaurant_rtree_index = create_rtree_index(restaurant_data)
nearest_restaurants_bf = best_first_nearest_neighbors(user_queries, restaurant_rtree_index)
print("\nNearest Restaurants using Best First Algorithm with R-tree:")
for result in nearest_restaurants_bf:
    print(f"User {result[0]} -> Restaurant {result[1]} (Distance: {result[2]:.2f})")

# BF with Divide-and-Conquer
nearest_restaurants_bf_dc = bf_divide_and_conquer_nearest_neighbors(user_queries, restaurant_data)
print("\nNearest Restaurants using BF with Divide-and-Conquer:")
for result in nearest_restaurants_bf_dc:
    print(f"User {result[0]} -> Restaurant {result[1]} (Distance: {result[2]:.2f})")


Nearest Restaurants using Sequential Scan:
User 1 -> Restaurant 121862 (Distance: 0.12)
User 2 -> Restaurant 12827 (Distance: 0.10)
User 3 -> Restaurant 113975 (Distance: 0.08)
User 4 -> Restaurant 149159 (Distance: 0.12)
User 5 -> Restaurant 94300 (Distance: 0.09)
User 6 -> Restaurant 43818 (Distance: 0.15)
User 7 -> Restaurant 8816 (Distance: 0.11)
User 8 -> Restaurant 79795 (Distance: 0.07)
User 9 -> Restaurant 4586 (Distance: 0.14)
User 10 -> Restaurant 67804 (Distance: 0.18)
User 11 -> Restaurant 11425 (Distance: 0.22)
User 12 -> Restaurant 85208 (Distance: 0.04)
User 13 -> Restaurant 100197 (Distance: 0.13)
User 14 -> Restaurant 58292 (Distance: 0.07)
User 15 -> Restaurant 64709 (Distance: 0.05)
User 16 -> Restaurant 97516 (Distance: 0.04)
User 17 -> Restaurant 74801 (Distance: 0.09)
User 18 -> Restaurant 58974 (Distance: 0.09)
User 19 -> Restaurant 93541 (Distance: 0.07)
User 20 -> Restaurant 34450 (Distance: 0.08)
User 21 -> Restaurant 89802 (Distance: 0.12)
User 22 -> Restaura

##  **Output:**  For each algorithm (Sequential Scan Based, BF Algorithm, and BF with Divideand-Conquer), display and output the following information in a single txt file:  The ID, x, and y coordinates of the nearest neighbor identified for each query point

In [ ]:
import os
import math
from time import time
from rtree import index

# Function to read data from a text file
def read_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            point_id = int(parts[0])
            x = float(parts[1])
            y = float(parts[2])
            data.append((point_id, x, y))
    return data

# Function to compute Euclidean distance between two points
def euclidean_distance(point1, point2):
    return math.sqrt((point1[1] - point2[1]) ** 2 + (point1[2] - point2[2]) ** 2)

# Function to find the nearest neighbor for each query point in a dataset using Sequential Scan
def sequential_scan_nearest_neighbors(queries, dataset):
    nearest_neighbors = []
    for query in queries:
        min_distance = float('inf')
        nearest_point = None
        for point in dataset:
            distance = euclidean_distance(query, point)
            if distance < min_distance:
                min_distance = distance
                nearest_point = point
        nearest_neighbors.append((query[0], nearest_point[0], nearest_point[1], nearest_point[2], min_distance))
    return nearest_neighbors

# Function to create an R-tree index from a dataset
def create_rtree_index(dataset):
    p = index.Property()
    idx = index.Index(properties=p)
    for point in dataset:
        idx.insert(point[0], (point[1], point[2], point[1], point[2]), obj=point)
    return idx

# Function to find the nearest neighbor for each query point using the Best First algorithm with R-tree
def best_first_nearest_neighbors(queries, rtree_index):
    nearest_neighbors = []
    for query in queries:
        nearest = list(rtree_index.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        distance = euclidean_distance(query, nearest)
        nearest_neighbors.append((query[0], nearest[0], nearest[1], nearest[2], distance))
    return nearest_neighbors

# Function to divide the dataset into two subspaces
def divide_dataset(dataset, dimension='x'):
    dataset_sorted = sorted(dataset, key=lambda point: point[1] if dimension == 'x' else point[2])
    mid = len(dataset_sorted) // 2
    return dataset_sorted[:mid], dataset_sorted[mid:]

# Function to find the nearest neighbor for each query point using BF with Divide-and-Conquer
def bf_divide_and_conquer_nearest_neighbors(queries, dataset):
    subspace1, subspace2 = divide_dataset(dataset)
    rtree_index1 = create_rtree_index(subspace1)
    rtree_index2 = create_rtree_index(subspace2)
    
    nearest_neighbors = []
    for query in queries:
        nearest1 = list(rtree_index1.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        nearest2 = list(rtree_index2.nearest((query[1], query[2], query[1], query[2]), 1, objects=True))[0].object
        distance1 = euclidean_distance(query, nearest1)
        distance2 = euclidean_distance(query, nearest2)
        if distance1 < distance2:
            nearest_neighbors.append((query[0], nearest1[0], nearest1[1], nearest1[2], distance1))
        else:
            nearest_neighbors.append((query[0], nearest2[0], nearest2[1], nearest2[2], distance2))
    return nearest_neighbors

# Paths to dataset files
restaurant_file = os.path.expanduser('~/mol/data/assignment-2-datasets-1 (1)/Task1_Datasets/restaurant_dataset.txt')
queries_file = os.path.expanduser('~/mol/data/assignment-2-datasets-1 (1)/Task1_Datasets/query_points.txt')

# Read datasets and queries
restaurant_data = read_dataset(restaurant_file)
user_queries = read_dataset(queries_file)

# Sequential Scan
start_time = time()
nearest_restaurants_seq = sequential_scan_nearest_neighbors(user_queries, restaurant_data)
total_time_seq = time() - start_time
average_time_seq = total_time_seq / len(user_queries)

# Best First with R-tree
restaurant_rtree_index = create_rtree_index(restaurant_data)
start_time = time()
nearest_restaurants_bf = best_first_nearest_neighbors(user_queries, restaurant_rtree_index)
total_time_bf = time() - start_time
average_time_bf = total_time_bf / len(user_queries)

# BF with Divide-and-Conquer
start_time = time()
nearest_restaurants_bf_dc = bf_divide_and_conquer_nearest_neighbors(user_queries, restaurant_data)
total_time_bf_dc = time() - start_time
average_time_bf_dc = total_time_bf_dc / len(user_queries)

# Write results to a text file
output_file = os.path.expanduser('~/mol/data/assignment-2-datasets-1 (1)/Task1_Datasets/nearest_neighbors_results.txt')

with open(output_file, 'w') as file:
    file.write("Nearest Restaurants using Sequential Scan:\n")
    for result in nearest_restaurants_seq:
        file.write(f"Query {result[0]} -> id={result[1]}, x={result[2]}, y={result[3]} (Distance: {result[4]:.2f})\n")
    file.write(f"Total running time: {total_time_seq:.2f} seconds\n")
    file.write(f"Average time per query: {average_time_seq:.4f} seconds\n\n")
    
    file.write("Nearest Restaurants using Best First Algorithm with R-tree:\n")
    for result in nearest_restaurants_bf:
        file.write(f"Query {result[0]} -> id={result[1]}, x={result[2]}, y={result[3]} (Distance: {result[4]:.2f})\n")
    file.write(f"Total running time: {total_time_bf:.2f} seconds\n")
    file.write(f"Average time per query: {average_time_bf:.4f} seconds\n\n")
    
    file.write("Nearest Restaurants using BF with Divide-and-Conquer:\n")
    for result in nearest_restaurants_bf_dc:
        file.write(f"Query {result[0]} -> id={result[1]}, x={result[2]}, y={result[3]} (Distance: {result[4]:.2f})\n")
    file.write(f"Total running time: {total_time_bf_dc:.2f} seconds\n")
    file.write(f"Average time per query: {average_time_bf_dc:.4f} seconds\n")

print("Results have been written to nearest_neighbors_results.txt")
